# High-Level Plan

## Game-level sim
- Kenpom implied wp for each team
- Give players avg reg season pts for each

## Tournament-level sim

- Treat each region & final four as trees
- Simulate whole tournament n times
- For each game, use kenpom implied wp to determine winner
- Keep track of each player's estimated pts

## Best combo selection
- For each player, calculate a distribution of pts
- Naively use greedy algorithm selected the top 15 players
- More sophisticated approach: select best *set* of 15 players

## Backtesting
- Use historical data to see how well the model would have performed

---

In [84]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from collections import deque

## Game-level (kenpom)

In [197]:
def scrape_kenpom_to_df():
    """
    Scrape KenPom data and return it as a pandas DataFrame.
    
    Returns:
        pd.DataFrame: DataFrame containing KenPom data.
    """
    url = "https://kenpom.com/index.php?y=2024"
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(response)
        raise Exception(f"Failed to fetch data from {url}")
    
    soup = BeautifulSoup(response.text, 'html.parser')
    # Parse the HTML content using BeautifulSoup
    table = soup.find("table", id ="ratings-table")

    if table is None:
        raise Exception("Failed to find the ratings table on the page.")
    
    kenpom_df = pd.read_html(str(table))[0]
    
    return kenpom_df

In [230]:
kenpom_df = scrape_kenpom_to_df()
kenpom_df

/var/folders/32/gv3pqqj908z_m6260t1ghrxr0000gn/T/ipykernel_63735/1674597044.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  kenpom_df = pd.read_html(str(table))[0]


Unnamed: 0_level_0      Unnamed: 1_level_0  Unnamed: 2_level_0  \
                     Rk                    Team                Conf   
     Unnamed: 0_level_2      Unnamed: 1_level_2  Unnamed: 2_level_2   
                     Rk                    Team                Conf   
     Unnamed: 0_level_4      Unnamed: 1_level_4  Unnamed: 2_level_4   
                     Rk                    Team                Conf   
     Unnamed: 0_level_6      Unnamed: 1_level_6  Unnamed: 2_level_6   
                     Rk                    Team                Conf   
     Unnamed: 0_level_8      Unnamed: 1_level_8  Unnamed: 2_level_8   
                     Rk                    Team                Conf   
    Unnamed: 0_level_10     Unnamed: 1_level_10 Unnamed: 2_level_10   
                     Rk                    Team                Conf   
    Unnamed: 0_level_12     Unnamed: 1_level_12 Unnamed: 2_level_12   
                     Rk                    Team                Conf   
    Unnamed: 0_level_14     Unnamed: 1_level_14 Unnamed: 2_level_14   
                     Rk                    Team                Conf   
    Unnamed: 0_level_16     Unnamed: 1_level_16 Unnamed: 2_level_16   
                     Rk                    Team                Conf   
    Unnamed: 0_level_18     Unnamed: 1_level_18 Unnamed: 2_level_18   
                     Rk                    Team                Conf   
0                     1           Connecticut 1                  BE   
1                     2               Houston 1                 B12   
2                     3                Purdue 1                 B10   
3                     4                Auburn 4                 SEC   
4                     5             Tennessee 2                 SEC   
..                  ...                     ...                 ...   
375                 360                   IUPUI                Horz   
376                 NaN                     NaN                 NaN   
377                  Rk                    Team                Conf   
378                 361              Coppin St.                MEAC   
379                 362  Mississippi Valley St.                SWAC   

     Unnamed: 3_level_0  Unnamed: 4_level_0  Unnamed: 5_level_0  \
                    W-L              NetRtg                ORtg   
     Unnamed: 3_level_2  Unnamed: 4_level_2  Unnamed: 5_level_2   
                    W-L              NetRtg                ORtg   
     Unnamed: 3_level_4  Unnamed: 4_level_4  Unnamed: 5_level_4   
                    W-L              NetRtg                ORtg   
     Unnamed: 3_level_6  Unnamed: 4_level_6  Unnamed: 5_level_6   
                    W-L              NetRtg                ORtg   
     Unnamed: 3_level_8  Unnamed: 4_level_8  Unnamed: 5_level_8   
                    W-L              NetRtg                ORtg   
    Unnamed: 3_level_10 Unnamed: 4_level_10 Unnamed: 5_level_10   
                    W-L              NetRtg                ORtg   
    Unnamed: 3_level_12 Unnamed: 4_level_12 Unnamed: 5_level_12   
                    W-L              NetRtg                ORtg   
    Unnamed: 3_level_14 Unnamed: 4_level_14 Unnamed: 5_level_14   
                    W-L              NetRtg                ORtg   
    Unnamed: 3_level_16 Unnamed: 4_level_16 Unnamed: 5_level_16   
                    W-L              NetRtg                ORtg   
    Unnamed: 3_level_18 Unnamed: 4_level_18 Unnamed: 5_level_18   
                    W-L              NetRtg                ORtg   
0                  37-3              +36.43               127.5   
1                  32-5              +31.17               118.9   
2                  34-5              +30.62               125.2   
3                  27-8              +27.99               120.4   
4                  27-9              +26.61               116.8   
..                  ...                 ...                 ...   
375                6-26              -25.61                92

In [231]:
kenpom_df.columns = kenpom_df.columns.get_level_values(1)

In [233]:
# select only the relevant columns
kenpom_df = kenpom_df.iloc[:, :5]

# get rid of duplicate rows of header
kenpom_df = kenpom_df.drop_duplicates()

# drop extra row with header
kenpom_df.dropna(subset=['Team'], inplace=True)

# reset index after drops
kenpom_df.reset_index(drop=True, inplace=True)

# drop extra copy of column labels
kenpom_df.drop(kenpom_df[kenpom_df['Team'] == 'Team'].index, inplace=True)

# remove seed numbers
kenpom_df['Team'] = kenpom_df['Team'].str.replace(r' \d{1,2}', '', regex=True)

kenpom_df

,Rk,Team,Conf,W-L,NetRtg
0,1,Connecticut,BE,37-3,+36.43
1,2,Houston,B12,32-5,+31.17
2,3,Purdue,B10,34-5,+30.62
3,4,Auburn,SEC,27-8,+27.99
4,5,Tennessee,SEC,27-9,+26.61
...,...,...,...,...,...
357,358,Pacific,WCC,6-26,-22.84
358,359,Stonehill,NEC,4-27,-22.93
359,360,IUPUI,Horz,6-26,-25.61
360,361,Coppin St.,MEAC,2-27,-25.73


In [ ]:
def wp_kenpom(team1, team2, ratings_df, sd=11):
    # Extract ratings for each team in the matchup
    # First, check if team name is in the kenpom teams
    if team1.team_name not in kenpom_df['Team'].values:
        team1_kp = sr_to_kenpom[team1.team_name]
    else:
        team1_kp = team1.team_name
    
    if team2.team_name not in kenpom_df['Team'].values:
        team2_kp = sr_to_kenpom[team2.team_name]
    else:
        team2_kp = team2.team_name
        
    rating1 = ratings_df[ratings_df['Team'] == team1_kp]['NetRtg'].values[0]
    rating2 = ratings_df[ratings_df['Team'] == team2_kp]['NetRtg'].values[0]
    rating_diff = float(rating1) - float(rating2)

    # Calculate the probability of team1 winning
    prob_team1_wins = 1 / (1 + np.exp(-rating_diff / sd))

    return prob_team1_wins

def simulate_game_kenpom(team1, team2, ratings_df):
    # Calculate the probability of team1 winning
    prob_team1_wins = wp_kenpom(team1, team2, ratings_df)

    # Simulate the game based on the probability
    if np.random.rand() < prob_team1_wins:
        return team1
    else:
        return team2


## Parse Old Results from SportsReference

Need to come up with a score for a certain strategy. Would also be nice to represent the uncertainty somehow.

## Scraping

In [72]:
def read_unplayed_tournament(url, region):
    # Use requests to get the content of the webpage
    response = requests.get(url)
    html_content = response.text

    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the container for the east region
    east_round_64 = soup.find(id=region).find(class_='team16').find(class_='round', recursive=False)
    matchups = []

    games = east_round_64.find_all('div', recursive=False)  # Each game is contained in a div directly under the round div
    for game in games:
        teams = game.find_all('div', recursive=False)
        if teams[0].find('span', recursive=False) is not None:
            if game.find('span', recursive=False) is not None: # regular games
                location = game.find('span', recursive=False).text.strip()[3:]
                team1_seed, team1_name, team1_link = teams[0].find('span').text, teams[0].find('a').text, teams[0].find('a')['href']
                team2_seed, team2_name, team2_link = teams[1].find('span').text, teams[1].find('a').text, teams[1].find('a')['href']
            else: # play-in games
                location = "TBD"
                team1_seed, team1_name, team1_link = teams[0].find('span').text, teams[0].find('a').text, teams[0].find('a')['href']
                team2_seed = 16 - (int(team1_seed) - 1)
                team2_name = "Play-In"
                team2_link = None

            matchups.append({
                'team_1': {'seed': team1_seed, 'name': team1_name, 'link': team1_link},
                'team_2': {'seed': team2_seed, 'name': team2_name, 'link': team2_link},
                'location': location
            })

    return matchups

In [73]:
BASE_URL = "https://www.sports-reference.com/cbb/postseason/men/{}-ncaa.html"

In [74]:
year = 2024
url = BASE_URL.format(year)

regions = ["east", "west", "south", "midwest"]
matchups = read_unplayed_tournament(url, "west")
for matchup in matchups:
    print(matchup)

{'team_1': {'seed': '1', 'name': 'UNC', 'link': '/cbb/schools/north-carolina/men/2024.html'}, 'team_2': {'seed': '16', 'name': 'Wagner', 'link': '/cbb/schools/wagner/men/2024.html'}, 'location': 'Charlotte, NC'}
{'team_1': {'seed': '8', 'name': 'Mississippi State', 'link': '/cbb/schools/mississippi-state/men/2024.html'}, 'team_2': {'seed': '9', 'name': 'Michigan State', 'link': '/cbb/schools/michigan-state/men/2024.html'}, 'location': 'Charlotte, NC'}
{'team_1': {'seed': '5', 'name': "Saint Mary's", 'link': '/cbb/schools/saint-marys-ca/men/2024.html'}, 'team_2': {'seed': '12', 'name': 'Grand Canyon', 'link': '/cbb/schools/grand-canyon/men/2024.html'}, 'location': 'Spokane, WA'}
{'team_1': {'seed': '4', 'name': 'Alabama', 'link': '/cbb/schools/alabama/men/2024.html'}, 'team_2': {'seed': '13', 'name': 'College of Charleston', 'link': '/cbb/schools/college-of-charleston/men/2024.html'}, 'location': 'Spokane, WA'}
{'team_1': {'seed': '6', 'name': 'Clemson', 'link': '/cbb/schools/clemson/me

In [ ]:
print(read_unplayed_tournament(BASE_URL.format(year), "east"))
print(read_unplayed_tournament(BASE_URL.format(year), "west"))
print(read_unplayed_tournament(BASE_URL.format(year), "south"))
print(read_unplayed_tournament(BASE_URL.format(year), "midwest"))

## Data Structures for Simulation

The bracket is a big binary tree of games (the championship game is root node).

We populate the tree by taking a list of initial matchups and creating unique nodes (i.e. TournamentGame objects) for them. Matchups that play each other next will have the same parent TournamentGame node. We can create a queue of simulated games to process the games to be simulated. When a game gets both opponents filled, it will be popped off the queue and simulated.

Could model this with classes or just with a giant dataframe. Classes seems cleaner.

In [270]:
class Team:
    def __init__(self, team_name, seed, roster):
        self.team_name = team_name
        self.seed = seed
        self.roster = roster

    def __str__(self):
        return f"{self.team_name} ({self.seed})"

class Matchup:
    def __init__(self, team1, team2, location=None):
        self.team1 = team1
        self.team2 = team2
        self.location = location

    @classmethod
    def parse_matchup(cls, matchup_dict):
        """
        Parses a matchup string in the format "Team1 vs Team2" and returns a Matchup object.
        """
        t1 = Team(matchup_dict['team_1']['name'], matchup_dict['team_1']['seed'], matchup_dict['team_1']['link'])
        t2 = Team(matchup_dict['team_2']['name'], matchup_dict['team_2']['seed'], matchup_dict['team_2']['link'])
        
        return cls(t1, t2, matchup_dict['location'])

class Node:
    def __init__(self, matchup=None, left=None, right=None, parent=None):
        self.matchup = matchup
        self.left = left
        self.right = right
        self.parent = parent
        self.winner = None  # Winning team from this matchup

    def is_leaf(self):
        return self.left is None and self.right is None

class Region:
    def __init__(self, matchups):
        self.matchup_q = deque(self.parse_matchups_into_nodes(matchups))
        self.championship = None

    def parse_matchups_into_nodes(self, matchups_list):
        '''
        Takes a list of matchups and creates a tree of matchups.
        '''
        nodes = []
        for matchup in matchups_list:
            # Parse the matchup and create a Matchup object
            Matchup_obj = Matchup.parse_matchup(matchup)
            # Create a Node object for the matchup
            node = Node(matchup=Matchup_obj)
            nodes.append(node)
        
        return nodes

    def sim_region(self):
        '''
        Starting from initial matchups, sims a region of the tournament.
        '''
        # east plays west and south plays midwest in final four, so process east, west, south, midwest in that order
        # could start simulations in parallel when you add games to the queue
        while len(self.matchup_q) >= 2:
            game1 = self.matchup_q.popleft()  # pop the first game
            game2 = self.matchup_q.popleft()  # pop the second game

            # simulate those two games and do appropriate calculation and bookkeeping
            # uncertainty could be here in terms of player score (use reg. season to make distribution?)
            # simulation will lead to uncertainty in game outcomes

            
            # game1winner = np.random.choice([game1.matchup.team1, game1.matchup.team2], p=[0.5, 0.5])
            game1winner = simulate_game_kenpom(game1.matchup.team1, game1.matchup.team2, kenpom_df)
            game1.winner = game1winner

            # game2winner = np.random.choice([game2.matchup.team1, game2.matchup.team2], p=[0.5, 0.5])
            game2winner = simulate_game_kenpom(game2.matchup.team1, game2.matchup.team2, kenpom_df)
            game2.winner = game2winner
                
            # create new Game object with the winner of those two games
            new_game = Node(matchup=Matchup(game1winner, game2winner))
            
            # set the left and right children of the new game
            new_game.left = game1
            new_game.right = game2
            
            # mark the game as the parent of the other two games
            game1.parent = new_game
            game2.parent = new_game

            # add the new game to queue
            self.matchup_q.append(new_game)
        
        # sim championship
        if len(self.matchup_q) == 1:
            final_game = self.matchup_q.popleft()
            champ = np.random.choice([final_game.matchup.team1, final_game.matchup.team2], p=[0.5, 0.5])
            
            final_game.winner = champ
        
            self.championship = final_game
        else:
            raise Exception("Error: Region simulation did not end with a single championship game.")
    
    def print_region(self):
        '''
        Prints the bracket in a readable format.
        '''
        def print_node(node, indent=""):
            if node is None:
                return
            
            green_color = "\033[92m"
            reset_color = "\033[0m"
                
            # Assume each node has a 'winner' attribute. Fall back to a placeholder if not.
            winner = getattr(node, "winner", "TBD")
            
            # If the node has children, print them in a bracket-like structure.
            if node.is_leaf():
                # Print the matchup details
                t1 = node.matchup.team1
                t2 = node.matchup.team2

                if node.winner == node.matchup.team1:
                    print(indent + "├──" + f"{green_color}{node.matchup.team1}{reset_color}")
                    print(indent + "└──" + str(node.matchup.team2))
                else:
                    print(indent + "├──" + str(node.matchup.team1))
                    print(indent + "└──" + f"{green_color}{node.matchup.team2}{reset_color}")
           
            else:
                # find winner of the game
                if node.winner == node.matchup.team1:
                    print(indent + "├──" + f"{green_color}{node.matchup.team1}{reset_color}")
                    print_node(node.left, indent + "│   ")
                    print(indent + "└──" + str(node.matchup.team2))
                    print_node(node.right, indent + "    ")
                else:
                    print(indent + "├──" + str(node.matchup.team1))
                    print_node(node.left, indent + "│   ")
                    print(indent + "└──" + f"{green_color}{node.matchup.team2}{reset_color}")
                    print_node(node.right, indent + "    ")
                
        print(self.championship.winner)
        print_node(self.championship)

class Tournament:        
    def __init__(self, east_matchups, west_matchups, south_matchups, midwest_matchups):
        self.east = east_matchups
        self.west = west_matchups
        self.south = south_matchups
        self.midwest = midwest_matchups
        self.players_bookkeeping = None

    def simulate_tournament(self):
       for region in [self.east, self.west, self.south, self.midwest]:
            region.championship = region.sim_region()
            region.print_region()

    def bracket_distance(team1, team2):
        '''
        How many games it will take before team1 plays team2 if they keep winning.
        '''
        pass

In [278]:
from collections import defaultdict

east_2024_list = read_unplayed_tournament(BASE_URL.format(2024), "east")

def simulate_n_tournaments(N=100):
    champions = defaultdict(int)

    for _ in range(N):
        east_2024 = Region(east_2024_list)
        east_2024.sim_region()  # Assume you have this method
        champ = east_2024.championship.winner # winner of championship
        champions[str(champ)] += 1

    # Convert to probabilities
    champion_probs = {team: count / N for team, count in champions.items()}
    return champion_probs

In [279]:
simulate_n_tournaments(1000)

{'San Diego State (5)': 0.04,
 'UConn (1)': 0.314,
 'Iowa State (2)': 0.185,
 'Auburn (4)': 0.109,
 'Illinois (3)': 0.17,
 'Florida Atlantic (8)': 0.014,
 'Northwestern (9)': 0.017,
 'BYU (6)': 0.08,
 'Washington State (7)': 0.036,
 'Drake (10)': 0.02,
 'UAB (12)': 0.002,
 'South Dakota State (15)': 0.003,
 'Morehead State (14)': 0.002,
 'Duquesne (11)': 0.006,
 'Yale (13)': 0.002}

In [ ]:
east_2024.sim_region()
east_2024.print_region()

San Diego State (5)
├──San Diego State (5)
│   ├──Florida Atlantic (8)
│   │   ├──Stetson (16)
│   │   │   ├──UConn (1)
│   │   │   └──Stetson (16)
│   │   └──Florida Atlantic (8)
│   │       ├──Florida Atlantic (8)
│   │       └──Northwestern (9)
│   └──San Diego State (5)
│       ├──San Diego State (5)
│       │   ├──San Diego State (5)
│       │   └──UAB (12)
│       └──Yale (13)
│           ├──Auburn (4)
│           └──Yale (13)
└──Illinois (3)
    ├──Illinois (3)
    │   ├──BYU (6)
    │   │   ├──BYU (6)
    │   │   └──Duquesne (11)
    │   └──Illinois (3)
    │       ├──Illinois (3)
    │       └──Morehead State (14)
    └──Iowa State (2)
        ├──Washington State (7)
        │   ├──Washington State (7)
        │   └──Drake (10)
        └──Iowa State (2)
            ├──Iowa State (2)
            └──South Dakota State (15)


In [249]:
sr_to_kenpom = {
    'Brigham Young': 'BYU',
    'Saint Mary\'s (CA)': 'Saint Mary\'s',
    'St. John\'s (NY)': 'St. John\'s',
    'Pittsburgh': 'Pittsburgh',
    'Mississippi State': 'Mississippi St.',
    'Central Florida': 'UCF',
    'Virginia Commonwealth': 'VCU',
    'Southern California': 'USC',
    'Mississippi': 'Ole Miss',
    'Massachusetts': 'UMass',
    'Miami (FL)': 'Miami FL',
    'Nevada-Las Vegas': 'UNLV',
    'North Carolina State': 'N.C. State',
    'Louisiana State': 'LSU',
    'Connecticut': 'Connecticut',
    'North Carolina': 'North Carolina',
    'Pittsburgh': 'Pittsburgh',
    'St. John\'s (NY)': 'St. John\'s',
    'Louisiana State': 'LSU',
    'Central Florida': 'UCF',
    'Texas A&M-Corpus Christi': 'Texas A&M Corpus Chris',
    'Purdue Fort Wayne': 'Purdue Fort Wayne',
    'Loyola (IL)': 'Loyola Chicago',
    'College of Charleston': 'Charleston',
    'Loyola (IL)': 'Loyola Chicago',
    'Saint Joseph\'s': 'Saint Joseph\'s',
    'Saint Francis (PA)': 'Saint Francis',
    'Arkansas-Pine Bluff': 'Arkansas Pine Bluff',
    'SIU Edwardsville': 'SIU Edwardsville',
    'Maryland-Baltimore County': 'UMBC',
    'Maryland-Eastern Shore': 'Maryland Eastern Shore',
    'Massachusetts-Lowell': 'UMass Lowell',
    'Charleston Southern': 'Charleston Southern',
    'East Tennessee State': 'East Tennessee St.',
    'Florida International': 'FIU',
    'Loyola (IL)': 'Loyola Chicago',
    'Albany (NY)': 'Albany',
    'College of Charleston': 'Charleston',
    'Detroit Mercy': 'Detroit Mercy',
    'LIU': 'LIU',
    'North Carolina State': 'N.C. State',
    'Southeast Missouri State': 'Southeast Missouri St.',
    'Louisiana-Monroe': 'Louisiana Monroe',
    'Nicholls State': 'Nicholls St.',
    'Middle Tennessee': 'Middle Tennessee',
    'UMass Lowell': 'UMass Lowell',
    'UNC Asheville': 'UNC Asheville',
    'UNC Greensboro': 'UNC Greensboro',
    'UNC Wilmington': 'UNC Wilmington',
    'Texas-Rio Grande Valley': 'UT Rio Grande Valley',
    'Western Carolina': 'Western Carolina',
    'Northern Illinois': 'Northern Illinois',
    'Southern Illinois': 'Southern Illinois',
    'Fairleigh Dickinson': 'Fairleigh Dickinson',
    'Alcorn State': 'Alcorn St.',
    'Cal State Bakersfield': 'Cal St. Bakersfield',
    'Cal State Fullerton': 'Cal St. Fullerton',
    'Cal State Northridge': 'Cal St. Northridge',
    'Prairie View': 'Prairie View A&M',
    'Southern Methodist': 'SMU',
    'Southern Mississippi': 'Southern Miss',
    'East Tennessee State': 'East Tennessee St.',
    'Eastern Illinois': 'Eastern Illinois',
    'Eastern Kentucky': 'Eastern Kentucky',
    'Eastern Michigan': 'Eastern Michigan',
    'Eastern Washington': 'Eastern Washington',
    'Illinois-Chicago': 'Illinois Chicago',
    'Kansas City': 'UMKC',
    'Louisiana-Monroe': 'Louisiana Monroe',
    'Maryland-Eastern Shore': 'Maryland Eastern Shore',
    'Tennessee-Martin': 'Tennessee Martin',
    'Texas Southern': 'Texas Southern',
    'North Dakota State': 'North Dakota St.',
    'Southeast Missouri State': 'Southeast Missouri St.',
    'Virginia Commonwealth': 'VCU',
    'UConn': 'Connecticut',
    'San Diego State': 'San Diego St.',
    'Morehead State': 'Morehead St.',
    'Washington State': 'Washington St.',
    'Iowa State': 'Iowa St.',
    'South Dakota State': 'South Dakota St.'
}

In [ ]:
east_2024 = Region(read_unplayed_tournament(BASE_URL.format(2024), "east"))
west_2024 = Region(read_unplayed_tournament(BASE_URL.format(2024), "west"))
south_2024 = Region(read_unplayed_tournament(BASE_URL.format(2024), "south"))
midwest_2024 = Region(read_unplayed_tournament(BASE_URL.format(2024), "midwest"))

# Create a Tournament object
tournament = Tournament(east_2024, west_2024, south_2024, midwest_2024)
# Simulate the tournament
tournament.simulate_tournament()

In [166]:
def create_tournament_tree(year):
    q = []
    # add initial games to queue (probably will need special case for play-in games)
    # east plays west and south plays midwest in final four, so process east, west, south, midwest in that order
    print(read_unplayed_tournament(BASE_URL.format(year), "east"))
    print(read_unplayed_tournament(BASE_URL.format(year), "west"))
    print(read_unplayed_tournament(BASE_URL.format(year), "south"))
    print(read_unplayed_tournament(BASE_URL.format(year), "midwest"))
    

    # could start simulations in parallel when you add games to the queue
    while len(q) >= 2:
        # pop off two games from the queue
        # simulate those two games and do appropriate calculation and bookkeeping
            # uncertainty could be here in terms of player score (use reg. season to make distribution?)
            # simulation will lead to uncertainty in game outcomes
        # create new Game object with the winner of those two games
        # mark the game as the parent of the other two games
        # add the new game to queue
        pass

In [ ]:
create_tournament_tree(2024)

[{'team_1': {'seed': '1', 'name': 'UConn', 'link': '/cbb/schools/connecticut/men/2024.html'}, 'team_2': {'seed': '16', 'name': 'Stetson', 'link': '/cbb/schools/stetson/men/2024.html'}, 'location': 'Brooklyn, NY'}, {'team_1': {'seed': '8', 'name': 'Florida Atlantic', 'link': '/cbb/schools/florida-atlantic/men/2024.html'}, 'team_2': {'seed': '9', 'name': 'Northwestern', 'link': '/cbb/schools/northwestern/men/2024.html'}, 'location': 'Brooklyn, NY'}, {'team_1': {'seed': '5', 'name': 'San Diego State', 'link': '/cbb/schools/san-diego-state/men/2024.html'}, 'team_2': {'seed': '12', 'name': 'UAB', 'link': '/cbb/schools/alabama-birmingham/men/2024.html'}, 'location': 'Spokane, WA'}, {'team_1': {'seed': '4', 'name': 'Auburn', 'link': '/cbb/schools/auburn/men/2024.html'}, 'team_2': {'seed': '13', 'name': 'Yale', 'link': '/cbb/schools/yale/men/2024.html'}, 'location': 'Spokane, WA'}, {'team_1': {'seed': '6', 'name': 'BYU', 'link': '/cbb/schools/brigham-young/men/2024.html'}, 'team_2': {'seed': '